In [142]:
from DataCollections import sequence_length,no_sequences,actions
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical,
import os
import tensorflow as tf
import numpy as np

label_map = {label: num for num, label in enumerate(actions)}

DATA_PATH = os.path.join('E:\\9th sem\\MS_Thesis\\Awiros\\Project\\data\\MP_DATA')

def load_data():
    sequences, labels = [], []
    for action in actions:
        for sequence in range(no_sequences):
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])
    return np.array(sequences), labels

X, Y = load_data()

In [143]:
y = to_categorical(Y).astype(int)

In [144]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


In [145]:
X_train.shape

(85, 30, 1662)

## Build and Train model


In [175]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [177]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [178]:
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs["categorical_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True
callback = MyThresholdCallback(threshold=0.95)

In [179]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [180]:
model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [181]:
model.fit(X_train,y_train, epochs=2000, callbacks=[tb_callback,callback],validation_split=0.02)

Epoch 1/2000
3/3 [==============================] - 14s 4s/step - loss: 1.6087 - categorical_accuracy: 0.3373 - val_loss: 1.0914 - val_categorical_accuracy: 0.5000
Epoch 2/2000
3/3 [==============================] - 0s 165ms/step - loss: 4.4652 - categorical_accuracy: 0.3133 - val_loss: 2.4604 - val_categorical_accuracy: 0.0000e+00
Epoch 3/2000
3/3 [==============================] - 0s 144ms/step - loss: 4.8534 - categorical_accuracy: 0.2892 - val_loss: 3.0390 - val_categorical_accuracy: 0.0000e+00
Epoch 4/2000
3/3 [==================

In [78]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 30, 64)            442112    
_________________________________________________________________
lstm_22 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_23 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_21 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_22 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_23 (Dense)             (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

In [182]:
res=model.predict(X_test)


In [192]:
actions[np.argmax(res[4])]

'hello'

In [193]:
actions[np.argmax(y_test[4])]

'hello'

In [194]:
y_test

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [202]:
model.save('E:\\9th sem\\MS_Thesis\\Awiros\\Project\\models\\action.h5')

In [120]:
del model

NameError: name 'model' is not defined

In [203]:
model.load_weights('E:\\9th sem\\MS_Thesis\\Awiros\\Project\\models\\action.h5')

In [204]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [205]:
yhat = model.predict(X_train)

In [206]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

multilabel_confusion_matrix(ytrue, yhat)

array([[[52,  5],
        [ 2, 26]],

       [[56,  2],
        [ 5, 22]],

       [[55,  0],
        [ 0, 30]]], dtype=int64)

In [207]:
accuracy_score(ytrue, yhat)

0.9176470588235294